## Introducción al análisis de datos usando python y R

### Sesión 1 
#### 20 de abril, 2018. 

En esta sesión lo que haremos es usar la librería *tweepy* en python para acceder a Twitter y recolectar tuits desde el streaming filtrando por algunas palabras de nuestro interés.

Todo lo que haremos es seguir la documentación de *tweepy* y guardaremos el resultado del stream en un csv.

In [ ]:
# Primero instalamos la librería.
# Para hacer esto desde el notebook primero importamos sys

import sys

In [ ]:
# Ahora podemos instalar la librería usando el siguiente comando:

!{sys.executable} -m pip install tweepy

In [ ]:
# Las credenciales que obtuvimos al crear una aplicación en Tuiter las guardamos en un archivo credenciales.py
# Las importamos para usarlas con la librería:

import credenciales

In [ ]:
# Ahora importamos la librería tweepy y vamos a iniciar la autenticación. Tweepy soporta autenticación tipo oauth

import tweepy

authentication = tweepy.OAuthHandler(credenciales.consumer_key, credenciales.consumer_secret)
authentication.set_access_token(credenciales.access_token, credenciales.access_secret)

# La clase API permite el uso de los métodosde la API REST de tuiter 
# http://tweepy.readthedocs.io/en/v3.6.0/getting_started.html

api = tweepy.API(authentication)

In [ ]:
# Vamos a probar la librería
# Los ejemplos los obtenemos de la documentación: http://tweepy.readthedocs.io/en/v3.6.0/code_snippet.html

for tweet in tweepy.Cursor(api.user_timeline).items(10):
      print(tweet.text)

In [ ]:
# Ahora vamos a juntar algunas cuentas para recuperar información de Twitter

L=['ibm_mx', 'imate_unam', 'unam_mx' ] 

# Una cosa interesante que podemos recuperar es el número de seguidores usando followers_count

for nombre in L:
    usuario = api.get_user(nombre)
    print(usuario.screen_name, usuario.followers_count)

In [ ]:
# Ahora vamos a usar el streaming de Tuiter.

# http://tweepy.readthedocs.io/en/v3.6.0/streaming_how_to.html


import tweepy
#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        print(status.text)

In [ ]:
# Siguiendo la documentación, vemos que podemos crear un "escuchador de Tuiter" que va a manejar los tuits que vayan cayendo.
# También necesitaremos una librería para leer los datos que vienen en formato json
# Vamos a crear una carpeta llamada "datos" para ir guardando lo que recuperamos.

import json

from tweepy import Stream
from tweepy.streaming import StreamListener

class TwitterListener(StreamListener):

    def on_data(self,data):
        tweet=json.loads(str(data))
        print(tweet['text'])
        with open("./datos/streamin.json","a") as archivo:
            archivo.write(json.dumps(tweet))
            archivo.write('\n')
        return(True)

    def on_error(self, status):
        print(status)

try:
            twitter_stream = Stream(authentication, TwitterListener())
            twitter_stream.filter(track=['#Debate'], async=True) #En este punto ponemos las palabras que vamos a restrear
except:
            pass

In [ ]:
# La información quehemos obtenido la vamos a gurdar en un archivo csv
# algunas variables de interés son: el texto del tuit, el usuario y la fecha de creación
# pueden ir guardando más datos según los necesiten

import csv
import json

fields = ['created_at', 'user', 'text']

tweets = []
with open("datos/streamin.json") as infile:
    for line in infile:
        tweet = json.loads(line)
        information = []
        for field in fields:
            if field not in tweet:
                information.append('')
            elif field == 'user':
                information.append(tweet['user']['screen_name'])
                information.append(tweet['user']['id']) # Dentro de 'user' hay más información que solo el nombre del tuitero
            else:
                information.append(tweet[field])
        tweets.append(information)

with open("datos/streamin.csv", "w",encoding='utf-16') as outfile:
    csvwriter = csv.writer(outfile)
    csvwriter.writerow(['created_at', 'screen_name','id', 'text']) 
    csvwriter.writerows(tweets)

Con el csv guardado ya podemos iniciar nuestro análisis. Sin embargo, es importante resaltar que esta búsqueda es muy básica y la librería de *tweepy* tiene mucho más que ofrecernos.

También es importante recordar, habrá errores que de vez en cuando saldrán debido a las limitaciones de la API de tuiter y es importante tenerlas en cuenta cuando hagamos el rastreo de las campañas que nos interesan.

Todo lo que hemos hecho ha sido siguiendo la documentación de *Tweepy*, solo hemos agregado cuentas que nos interesan.
El paso a csv es común y pueden buscar varias referencias en la web.


#### Referencias extra

http://tweepy.readthedocs.io/en/v3.6.0/api.html#tweepy-error-exceptions

http://datacandy.github.io/warwick/datastream/index.html